# Exercise 01
The goal is to find the best set of hyper-parameters which
maximize the performance on a training set.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# This line is currently required to import
# HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from scipy.stats import expon, uniform
from scipy.stats import randint

df = pd.read_csv(
    "https://www.openml.org/data/get_csv/1595261/adult-census.csv")
# Or use the local copy: df = pd.read_csv('../datasets/adult-
# census.csv')

target_name = "class"
target = df[target_name].to_numpy()
data = df.drop(columns=target_name)

df_train, df_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

You should:
- create a preprocessor using an `OrdinalEncoder`
- use a `HistGradientBoostingClassifier` to make predictions
- use a `RandomizedSearchCV` to find the best set of hyper-
  parameters by tuning the following parameters: `learning_rate`,
  `l2_regularization`, `max_leaf_nodes`, and `min_samples_leaf`.

In [2]:
ordinal_encoding_columns = [
    'workclass', 'education', 'marital-status', 'occupation',
    'relationship', 'race', 'native-country', 'sex']

categories = [
    data[column].unique()
    for column in data[ordinal_encoding_columns]]

preprocessor = ColumnTransformer([
    ('ordinal-encoder', OrdinalEncoder(categories=categories),
     ordinal_encoding_columns)], remainder='passthrough',
                                 sparse_threshold=0)

model = Pipeline([
    ('preprocessor', preprocessor),
    ('gbrt', HistGradientBoostingClassifier(max_iter=50))])
param_distributions = {
    'gbrt__learning_rate': expon(loc=0.001, scale=0.5),
    'gbrt__l2_regularization': uniform(loc=0, scale=0.5),
    'gbrt__max_leaf_nodes': randint(5, 30),
    'gbrt__min_samples_leaf': randint(5, 30)}
model_grid_search = RandomizedSearchCV(
    model, param_distributions=param_distributions, n_iter=10,
    n_jobs=4)
model_grid_search.fit(df_train, target_train)
print(
    f"The accuracy score using a {model_grid_search.__class__.__name__} is "
    f"{model_grid_search.score(df_test, target_test):.2f}")
print(
    f"The best set of parameters is: {model_grid_search.best_params_}"
)

The accuracy score using a RandomizedSearchCV is 0.88
The best set of parameters is: {'gbrt__l2_regularization': 0.15238702139624238, 'gbrt__learning_rate': 0.22836744024996788, 'gbrt__max_leaf_nodes': 21, 'gbrt__min_samples_leaf': 16}


In [3]:
df_results = pd.DataFrame(model_grid_search.cv_results_)
columns = (['mean_test_score', 'std_test_score'] +
           [col for col in df_results.columns if 'param_' in col])
df_results.sort_values(by='mean_test_score',
                       ascending=False)[columns]

,mean_test_score,std_test_score,param_gbrt__l2_regularization,param_gbrt__learning_rate,param_gbrt__max_leaf_nodes,param_gbrt__min_samples_leaf
4,0.871202,0.001261,0.152387,0.228367,21,16
5,0.869810,0.002097,0.456043,0.177269,19,22
2,0.868882,0.002709,0.147098,0.382612,10,12
3,0.867899,0.001954,0.0865366,0.276309,8,19
9,0.865251,0.002257,0.12317,0.091077,15,13
7,0.865223,0.002431,0.0530611,0.50181,26,14
1,0.864732,0.002381,0.481884,0.60986,24,18
6,0.860746,0.003355,0.0187004,0.907443,18,9
8,0.857662,0.003125,0.432076,0.993723,17,6
0,0.855287,0.002175,0.173066,0.0966066,6,14
